## Importing Libraries and downloading data with pymongo, converting the clean query into a dataframe

In [376]:
from pymongo import MongoClient
import pandas as pd
import requests
import os
from dotenv import load_dotenv
load_dotenv()

client = MongoClient('mongodb://localhost:27017/')
db = client.GeoComps
a=db.GeoComps.find()
b=pd.DataFrame(a)
b["headquarters"].head()
b.shape

(2939, 10)

In [377]:
b.drop(columns=['_id'], inplace=True)
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75


### Choosing a radius of 2000 for my geoqueries, and testing the geopoint variable

In [378]:
radio=2000
geopoint=b["headquarters"][0]
geopoint

{'type': 'Point', 'coordinates': [-118.393064, 34.090368]}

## Creating a Find Near function to obtain data given a geopoint and a radius, testing that it works

In [379]:
def findNear(geopoint,radio):
     return list(db.GeoComps.find({
        "headquarters": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio,
         }
       }
    }))
geopoint=b["headquarters"][0]
lst=findNear(geopoint,radio)
lst

[{'_id': ObjectId('5d2759fa6497a5d8bff314d3'),
  'name': 'Geni',
  'lat': 34.090368,
  'lng': -118.393064,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.393064, 34.090368]},
  'number_of_employees': 18,
  'category_code': 'web',
  'founded_year': 2006,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31ba1'),
  'name': 'MyFreeImplants',
  'lat': 34.087825,
  'lng': -118.379493,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.379493, 34.087825]},
  'number_of_employees': 2,
  'category_code': 'network_hosting',
  'founded_year': 2005,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31b66'),
  'name': 'Meteor Games',
  'lat': 34.09316,
  'lng': -118.378335,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.378335, 34.09316]},
  'number_of_employees': 70,
  'category_code': 'games_video',
  'founded_year': 2007,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31d19'),
  'name': 'Book

## Creating a function to sum all the employees of companies that are near each companies, testing that it works

In [380]:
def sum_employees(lst):
    suma=0
    for i in lst:
        suma+= i["number_of_employees"]
    return suma

n=sum_employees(lst)
n


97

## Combining both function to obtain a list of the total number of employees near, appending this column to the original dataframe

In [381]:
sum_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=findNear(geopoint,radio)
    n=sum_employees(lost)
    sum_column.append(n)


In [382]:
b["Employees_Near"]=sum_column

In [383]:
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,97
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,2183
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35,180
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012


## Creating a function to obtain the ratio of New to Old companies nearby each firm

In [384]:
def new_old(lst):
    old=0
    new=0
    for i in lst:
        if i["age"]=='Old':
            old+=1
        else:
            new+=1
     
    if old>0:
        ratio= new/old
        return ratio
    else:
        return 1.5



## Using the funcition to get the ratios and appending the data to the dataframe

In [385]:
ratio_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=list(findNear(geopoint,radio))
    n=new_old(lost)
    ratio_column.append(n)


In [386]:
b["New/Old_Ratio"]=ratio_column

## I delete all the rows where the new ratio is 0, thus meaning there are no new companies in the area

In [387]:
b1=b[b['New/Old_Ratio'] != 0]
b1.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869,0.272727
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012,0.393939
5,Old,games_video,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,272,0.111111
6,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.0899512...",37.423390,-122.089951,Jajah,110,1234,0.090909
9,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-122.07948, ...",37.392936,-122.079480,Ustream,250,2065,0.166667


In [388]:
print(b1.shape)


(1685, 11)


## I see that almost 90% of the remaining firms are in the USA, so I focus on that country to avoid being to dispersed

In [389]:
b1['country'].value_counts().head()

USA    1124
GBR     115
CAN      88
FRA      55
DEU      50
Name: country, dtype: int64

In [390]:
b1=b1[b1['country'] == 'USA']
print(b1.shape)
b1.head()
b1.describe()

(1124, 11)


,founded_year,lat,lng,number_of_employees,Employees_Near,New/Old_Ratio
count,1124.000000,1124.000000,1124.000000,1124.000000,1124.000000,1124.000000
mean,2005.914591,38.477503,-102.759413,180.303381,3002.714413,0.684808
std,3.413842,4.481410,23.540114,3084.747416,10281.233931,0.538107
min,1990.000000,17.415429,-159.480262,1.000000,1.000000,0.047619
25%,2005.000000,37.320643,-122.265594,3.000000,41.000000,0.319444
50%,2007.000000,37.783662,-118.330006,9.000000,501.500000,0.428571
75%,2008.000000,40.747071,-77.231374,30.000000,2039.250000,1.000000
max,2013.000000,61.216583,78.434422,99999.000000,100961.000000,4.000000


## I see the statistics of the remaining data, I decide to keep the top25% of new-old ratio companies for now

In [391]:
new_old_upper=b1['New/Old_Ratio'].quantile(0.75)
print(new_old_upper)


1.0


In [392]:
b2=b1[b1['New/Old_Ratio'] >= new_old_upper]
b2.shape
b2.head(5)

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
16,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Mahalo,40,1197,1.166667
40,Old,network_hosting,USA,2005,"{'type': 'Point', 'coordinates': [-122.143701,...",37.425801,-122.143701,Box,950,1041,1.000000
66,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-105.276843,...",40.010492,-105.276843,IntenseDebate,4,46,1.000000
89,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-105.276843,...",40.010492,-105.276843,Filtrbox,14,46,1.000000
100,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.495025,...",34.018275,-118.495025,Docstoc,40,279,1.750000


## I also take the top 50% of companies w.r.t Number of NEAR employees

In [393]:
employee_median=b2['Employees_Near'].median()
print(employee_median)
b2=b2[b2['Employees_Near'] >= employee_median]
print(b2.shape)


13.0
(173, 11)


In [394]:
b3=b2.sort_values(['New/Old_Ratio', 'Employees_Near'], ascending=[False, False])
b3.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
1001,Old,games_video,USA,2006,"{'type': 'Point', 'coordinates': [-73.957968, ...",40.720764,-73.957968,VBS TV,100,130,4.0
2000,Old,software,USA,2000,"{'type': 'Point', 'coordinates': [-73.9586329,...",40.812933,-73.958633,Dot Com Infoway,500,519,3.0
178,Old,games_video,USA,2005,"{'type': 'Point', 'coordinates': [-83.734673, ...",42.292649,-83.734673,Zattoo,45,79,3.0
610,New,web,USA,2008,"{'type': 'Point', 'coordinates': [-83.731129, ...",42.282255,-83.731129,skoogO,1,79,3.0
1693,New,games_video,USA,2009,"{'type': 'Point', 'coordinates': [-83.7420579,...",42.280268,-83.742058,Mybandstock,19,79,3.0


## I sort by First the New_Old ratio in the near area and then the Number of Employees Near, creating a ranking with the new order

In [395]:
b4= b3.reset_index()
b4.head()
b5=b4.reset_index()
b5.head()
b6=b5.drop(columns='index')
b6['Rank']=b6['level_0'].rank()
b7=b6.drop(columns='level_0')


### I select the final top10 companies with the highest ranking

In [409]:
b8=b7[:9]

### I export this file to csv

In [397]:
 b8.to_csv('geo_clean.csv')

## From Tableau, I have selected an area in New York where at least 2 of the top companies are located close by. To select close to which of the two is the ideal location, I factor in which is close to a Starbucks. I get the Starbucks data from the Google Places API

In [398]:
key=os.environ['API_Token']
geocode_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=40.822087,-73.950677&radius=49900&keyword=Starbucks&key={}".format(key)
res = requests.get(geocode_url).json()



In [399]:
starb_lat=[]
starb_lng=[]
starb_name=[]
for i in res["results"]:
    starb_lat.append(i.get("geometry").get("location").get("lat"))
    starb_lng.append(i.get("geometry").get("location").get("lng"))
    starb_name.append(i.get("name"))



### I get the latitude and longitude of each starbucks in the selected area where the two finalist companies are located and create a dataframe

In [400]:

starbucks = pd.DataFrame()
starbucks['lat']  = starb_lat
starbucks['lng']  = starb_lng
starbucks['name']  = starb_name
starbucks.head()


,lat,lng,name
0,40.823359,-73.942575,Starbucks
1,40.815077,-73.959078,Starbucks
2,40.807803,-73.945136,Starbucks
3,40.823963,-73.930287,Starbucks
4,40.722553,-73.997942,Starbucks


In [401]:
b9=b8.drop(columns=['age','category_code','country','founded_year','headquarters','number_of_employees','Employees_Near','New/Old_Ratio','Rank'])

,lat,lng,name
0,40.720764,-73.957968,VBS TV
1,40.812933,-73.958633,Dot Com Infoway
2,42.292649,-83.734673,Zattoo
3,42.282255,-83.731129,skoogO
4,42.280268,-83.742058,Mybandstock


## I append both dataframes together to match the finalists with the nearby starbucks

In [404]:
s=starbucks.append(b9,ignore_index=True)

## I then export these locations to a csv 

In [408]:
s.to_csv('starbucks.csv')

# Finally: I conduct the final stage of my analysis in TABLEAU PUBLIC: PLEASE ACCESS THIS FINAL PART WITH THE FOLLOWING LINK

LINK: https://public.tableau.com/profile/pela1472#!/vizhome/CompaniesProject/Top10

## Thank you! :)